<a href="https://colab.research.google.com/github/KarineAyrs/science_work/blob/main/training/timm_image_models_efficient_b0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [ ]:
!pip install timm

In [3]:
import timm

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [ ]:
timm.list_models()

In [6]:
model_name = 'efficientnet_b0'

In [11]:
timm_model=timm.create_model(model_name=model_name)

timm_model.eval()

x= torch.randn(1, 3, 28, 28)
print(timm_model(x).shape)


torch.Size([1, 1000])


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [14]:
if torch.cuda.is_available():
    timm_model.cuda()

In [ ]:
learning_rate=0.001
batch_size=2
num_epochs=1

timm_model.train()

In [ ]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader=DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader=DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(timm_model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
  print(f'epoch:{epoch}')
  for batch, (data, targets) in enumerate(train_loader):
    # Get data to cuda if possible
    data=data.to(device=device)
    targets=targets.to(device=device)
    
    
    data = data.repeat(1,3,1,1)

    # forward
    scores = timm_model(data)
    loss = criterion(scores, targets)

    # backward
    optimizer.zero_grad()
    loss.backward()

    # gradient descent or adam step
    optimizer.step()



In [20]:

def check_accuracy(loader, model):
  if loader.dataset.train:
    print('Checking accuracy on training data')
  else:
    print('Checking accuracy o test data')
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x,y in loader:
      x=x.to(device=device)
      y=y.to(device=device)

      x=x.repeat(1,3,1,1)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct+=(predictions==y).sum()
      num_samples+=predictions.size(0)
    
    print(f'Got {num_correct}/{num_samples} with accuracy {(float(num_correct)/float(num_samples))*100}')
  
  model.train()
 
  

In [21]:
check_accuracy(train_loader, timm_model)
check_accuracy(test_loader, timm_model)

Checking accuracy on training data
Got 6457/60000 with accuracy 10.761666666666667
Checking accuracy o test data
Got 1107/10000 with accuracy 11.07
